In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
traindata=pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
testdata= pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])

In [ ]:
traindata.head()

In [ ]:
testdata.head()

In [ ]:
def pr(x):
   x=x.lower()
   tokens=nltk.word_tokenize(x)
   stop_words=set(stopwords.words('english'))
   punctuations = string.punctuation
   tagged_tokens = nltk.pos_tag(tokens)
   lemmatizer = WordNetLemmatizer()
   lemmas = []
   for token, tag in tagged_tokens:
     if token not in stop_words and token not in punctuations:
        pos = tag[0].lower() if tag[0].lower() in ['a', 'r', 'n', 'v'] else wordnet.NOUN
        lemma = lemmatizer.lemmatize(token, pos=pos)
        lemmas.append(lemma)

   filtered_sentence = ' '.join(lemmas)
   return filtered_sentence

In [ ]:
traindata['msg'] = traindata['msg'].apply(pr)
traindata.head()

In [ ]:
testdata['msg'] = testdata['msg'].apply(pr)
testdata.head()

In [ ]:
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
traindata['type'].value_counts().plot(kind='bar')
rus = RandomUnderSampler(sampling_strategy='majority')
X_resampled, y_resampled = rus.fit_resample(traindata[['msg']], traindata[['type']])

In [ ]:
y_resampled.value_counts().plot(kind='bar')


In [ ]:
x_train=X_resampled['msg']
y_train=y_resampled['type']
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_train

In [ ]:
y_train.value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(x_train)
X_tfidf=vectorizer.transform(x_train)
x=X_tfidf

In [ ]:
x.shape
x_train=x

In [ ]:
df_train_tfidf = pd.DataFrame.sparse.from_spmatrix(x_train, columns=vectorizer.vocabulary_.keys())

In [ ]:
label_mapper = lambda x: 1 if x == 'spam' else 0
testdata['type'] = testdata['type'].map(label_mapper)
x_test=testdata[['msg']]
y_test=testdata['type']

x_test

In [ ]:
X_tfidf_test = vectorizer.transform(testdata['msg'])
x_test=X_tfidf_test

In [ ]:
df_x_test_tfidf = pd.DataFrame.sparse.from_spmatrix(x_test, columns=vectorizer.vocabulary_.keys())

In [ ]:
x_train=df_train_tfidf
x_test=df_x_test_tfidf

In [ ]:
x_train

In [ ]:
x_test

In [ ]:
y_train.value_counts()

In [ ]:


print(x_train.shape)
print(y_train.shape)

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
clf = svm.SVC(kernel='linear')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_train)
y_testp = clf.predict(x_test)
acc_score = accuracy_score(y_train, y_pred)
acc_scoret = accuracy_score(y_test, y_testp)
print('Accuracy score train :', acc_score)
print('Accuracy score test :', acc_scoret)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):
  pred_text =[pr(pred_text)]
  X_tfidf_test = vectorizer.transform(pred_text)
  y_pred = clf.predict(X_tfidf_test)
  if y_pred<0.5:
    return [y_pred[0],'ham']
  else:
      return [y_pred[0],'spam']


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
